# Setting the model input data

Structure inside a dataframe the groundwater levels data and the input features accosiated to each well. 

Set the climate models data inside a dataframe.

In [19]:
%matplotlib notebook
import keras as ks
import pandas as pd
from functions import setinputdataset


import tensorflow as tf
import numpy as np
import os
import glob
import datetime
from scipy import stats
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
tf.random.set_seed(1)

In [9]:
rpath="D:/Data/students/mariana/data/" #Root path
#load dataframes saved as pickles files from previous pre-processing
datagw=pd.read_pickle(rpath+"/Pickle/GWfilldata.pkl")

datapr=pd.read_pickle(rpath+"/Pickle/datapr.pkl") #Change for the new information containing 505 wells
datatm=pd.read_pickle(rpath+"/Pickle/datatm.pkl")
datarh=pd.read_pickle(rpath+"/Pickle/datarh.pkl")

cmpr=pd.read_pickle(rpath+"/Pickle/cmprdf.pkl")
cmtm=pd.read_pickle(rpath+"/Pickle/cmtmdf.pkl")
cmrh=pd.read_pickle(rpath+"/Pickle/cmrhdf.pkl")

In [10]:
wellid="9610009"
#load datasets for the wellid
data=setinputdataset(wellid,datagw)
dfwell=data.setinputdata(datapr, datatm, datarh)
cmwelldf=data.setclimmodel(cmpr,cmtm,cmrh, modelname="MPI_WRF361H")


## CNN model

Convolutional Neural Networks (CNNs) are a type of neural network, primarly designed to manage image data

In [13]:
dfwell

,dates,GWL,pr,tm,rh
0,1972-05-31,0.340000,101.141808,12.010826,75.791420
1,1972-06-30,0.350000,114.478149,14.190649,76.898392
2,1972-07-31,0.370000,53.244888,17.778439,80.204132
3,1972-08-31,0.270000,69.392258,16.120594,78.927628
4,1972-09-30,0.210000,57.216351,11.917099,81.546867
...,...,...,...,...,...
519,2015-08-31,0.244072,78.257919,18.723993,78.075310
520,2015-09-30,0.350000,71.341843,14.133461,83.418434
521,2015-10-31,0.256884,31.189745,10.185218,87.224403
522,2015-11-30,0.460000,137.605682,8.945816,88.444527


In [56]:
#Train
tr=0.8
ytrain=dfwell[dfwell.columns[1]][0:round(tr*len(dfwell))]
xtrain=dfwell[dfwell.columns[2:]][0:round(tr*len(dfwell))]
#Validate
yval=dfwell[dfwell.columns[1]][round(tr*len(dfwell)):round(0.9*len(dfwell))]
xval=dfwell[dfwell.columns[2:]][round(tr*len(dfwell)):round(0.9*len(dfwell))]
#Test
ytest=dfwell[dfwell.columns[1]][round(0.9*len(dfwell)):]
xtest=dfwell[dfwell.columns[2:]][round(0.9*len(dfwell)):]

In [58]:
xtrain

,pr,tm,rh
0,101.141808,12.010826,75.791420
1,114.478149,14.190649,76.898392
2,53.244888,17.778439,80.204132
3,69.392258,16.120594,78.927628
4,57.216351,11.917099,81.546867
...,...,...,...
414,84.112366,9.010200,83.792206
415,57.279106,7.220184,85.782974
416,133.236038,6.490642,83.294113
417,70.641235,4.674838,87.399284


In [14]:

dataset = data[(data.index < GLOBAL_SETTINGS["test_start"])] #separate testdata

TrainingData = dataset[0:round(0.8 * len(dataset))]
StopData = dataset[round(0.8 * len(dataset))+1:round(0.9 * len(dataset))]
StopData_ext = dataset[round(0.8 * len(dataset))+1-GLOBAL_SETTINGS["seq_length"]:round(0.9 * len(dataset))] #extend data according to dealys/sequence length
OptData = dataset[round(0.9 * len(dataset))+1:]
OptData_ext = dataset[round(0.9 * len(dataset))+1-GLOBAL_SETTINGS["seq_length"]:] #extend data according to dealys/sequence length

TestData = data[(data.index >= GLOBAL_SETTINGS["test_start"]) & (data.index <= GLOBAL_SETTINGS["test_end"])] #testdata ccording to defined period
TestData_ext = pd.concat([dataset.iloc[-GLOBAL_SETTINGS["seq_length"]:], TestData], axis=0) # extend Testdata to be able to fill sequence later                                              



x, x_test, y, y_test = train_test_split(dfwell[dfwell.columns[2:]],dfwell[dfwell.columns[1]],
                                        test_size=0.2,train_size=0.8)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [26]:
x_train.shape[0]


314

In [44]:
y_train

325    0.24
419    0.39
387    0.25
308    0.49
420    0.32
       ... 
106    0.54
520    0.35
453    0.48
202    0.55
415    0.43
Name: GWL, Length: 314, dtype: float64

In [40]:
model= ks.Sequential()
#Filters-- small windows
# kernel_size -
#activation- relu is the most common one
model.add(ks.layers.convolutional.Conv1D(filters=64, kernel_size=3, 
                                         padding='same',
                                         activation='relu',
                                         input_shape=(x_train.shape[0],x_train.shape[1])))

model.add(ks.layers.convolutional.MaxPooling1D(padding='same'))
model.add(ks.layers.Flatten())
model.add(ks.layers.Dense(100, activation='relu'))
model.add(ks.layers.Dense(1, activation='linear'))


optimizer = tf.keras.optimizers.Adam(1e-3, epsilon=10e-3, clipnorm=True, clipvalue=1)
model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 314, 64)           640       
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 157, 64)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 10048)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 100)               1004900   
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 101       
Total params: 1,005,641
Trainable params: 1,005,641
Non-trainable params: 0
_________________________________________________________________


In [39]:

# early stopping
es = ks.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5)


325    0.24
419    0.39
387    0.25
308    0.49
420    0.32
       ... 
106    0.54
520    0.35
453    0.48
202    0.55
415    0.43
Name: GWL, Length: 314, dtype: float64

In [42]:

# fit network
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, verbose=0,
                    batch_size=128)



ValueError: in user code:

    E:\condaenvs\mgo\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    E:\condaenvs\mgo\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    E:\condaenvs\mgo\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    E:\condaenvs\mgo\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    E:\condaenvs\mgo\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    E:\condaenvs\mgo\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    E:\condaenvs\mgo\lib\site-packages\keras\engine\training.p

In [2]:
model= ks.Sequential()

GLOBAL_SETTINGS = {
    'pp': pp,
    'batch_size': batchsize_int, 
    'kernel_size': 3,
    'dense_size': densesize_int, 
    'filters': filters_int, 
    'seq_length': seqlength_int,
    'clip_norm': True,
    'clip_value': 1,
    'epochs': 30,
    'learning_rate': 1e-3,
}

In [4]:
model.add(ks.layers.convolutional.Conv1D(filters=GLOBAL_SETTINGS["filters"], kernel_size=GLOBAL_SETTINGS["kernel_size"], 
                                            padding='same', activation='relu', 
                                            input_shape=(GLOBAL_SETTINGS["seq_length"], X_train.shape[2])))
model.add(ks.layers.convolutional.MaxPooling1D(padding='same'))
model.add(ks.layers.Flatten())
model.add(ks.layers.Dense(GLOBAL_SETTINGS["dense_size"], activation='relu'))
model.add(ks.layers.Dense(1, activation='linear'))

In [5]:
model